In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

In [3]:
housing['data'][:3].round(2)

array([[ 8.3300e+00,  4.1000e+01,  6.9800e+00,  1.0200e+00,  3.2200e+02,
         2.5600e+00,  3.7880e+01, -1.2223e+02],
       [ 8.3000e+00,  2.1000e+01,  6.2400e+00,  9.7000e-01,  2.4010e+03,
         2.1100e+00,  3.7860e+01, -1.2222e+02],
       [ 7.2600e+00,  5.2000e+01,  8.2900e+00,  1.0700e+00,  4.9600e+02,
         2.8000e+00,  3.7850e+01, -1.2224e+02]])

In [4]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [5]:
scaler = StandardScaler()

In [6]:
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [7]:
X_train.shape

(11610, 8)

In [8]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [9]:
import tensorflow as tf
from tensorflow import keras
#from keras import layers
from keras.models import Model
# from keras.models import Model # this is same
from keras.layers import Dense, Input, concatenate
from keras import layers
from keras import models
                                                                # this concatenate is different from merge.Concatenate

/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/junweiluo/anaconda3/lib/python3.7/site-pac

In [10]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, input_B])  # this is a typo 
        # but proves not working if there is
        # no real concatenation.
        # easy to make mistakes.
        main_output = self.main_output(concat)
        aux_output = self.aux_ouput(hidden2)
        return main_output, aux_output
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
# not quite advisalbe, since it takes too long to develop and debug.
# easy to make mistakes

In [11]:
model = WideAndDeepModel(30, activation="relu")

In [12]:
model.summary()  # The model can only be summarized after fitting.

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [13]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

In [14]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [15]:
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 1s 81us/sample - loss: 2.2326 - output_1_loss: 1.9421 - output_2_loss: 4.8392 - val_loss: 1.1344 - val_output_1_loss: 0.8681 - val_output_2_loss: 3.5338
Epoch 2/10
11610/11610 [==============================] - 1s 44us/sample - loss: 1.0329 - output_1_loss: 0.8283 - output_2_loss: 2.8744 - val_loss: 0.8938 - val_output_1_loss: 0.7439 - val_output_2_loss: 2.2452
Epoch 3/10
11610/11610 [==============================] - 1s 48us/sample - loss: 0.8798 - output_1_loss: 0.7527 - output_2_loss: 2.0224 - val_loss: 0.7962 - val_output_1_loss: 0.6942 - val_output_2_loss: 1.7166
Epoch 4/10
11610/11610 [==============================] - 1s 50us/sample - loss: 0.8059 - output_1_loss: 0.7104 - output_2_loss: 1.6664 - val_loss: 0.7410 - val_output_1_loss: 0.6579 - val_output_2_loss: 1.4910
Epoch 5/10
11610/11610 [==============================] - 1s 46us/sample - loss: 0.7597 - output_1_loss: 0.

In [16]:
model.summary()
# no real output shape 
# no information how each layers connect to each other.
# can only get summary after fitting, since it is subclass.

Model: "wide_and_deep_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  210       
_________________________________________________________________
dense_1 (Dense)              multiple                  930       
_________________________________________________________________
dense_2 (Dense)              multiple                  36        
_________________________________________________________________
dense_3 (Dense)              multiple                  31        
Total params: 1,207
Trainable params: 1,207
Non-trainable params: 0
_________________________________________________________________


In [17]:
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

5160/5160 [==============================] - 0s 35us/sample - loss: 0.6245 - output_1_loss: 0.5570 - output_2_loss: 1.2201


In [18]:
total_loss  # to compare to validation result

0.6244786716708841

In [19]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

In [20]:
y_pred_main, y_pred_aux

(array([[0.26404145],
        [1.9094057 ],
        [5.232763  ]], dtype=float32), array([[1.4296337],
        [1.9811885],
        [2.519439 ]], dtype=float32))

In [21]:
model.save("my_keras_model.h5")  # see error message below.  Can't save the model, but the weights.

NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.

In [22]:
model.save_weights("my_weights.h5")

In [ ]:
keras.optimizers.  # check all availabe optimizers.

In [ ]:
keras.losses.    # check all available loss functions.

In [ ]:
keras.metrics.  # check all available metrics.